First break picking problem

First-break picking is the task of determining, given a set of raw seismic traces, the onsets of the first signal arrivals as accurately as possible. The accurate determination of the first arrivals onset first-break times is needed for calculating the static corrections, a fundamental stage of seismic data processing.

    Datasets
    Model architecture
    Training
    Inference
    Model evaluation
    Running time
    Criticism
    Summary
    Suggestions for improvement



In [ ]:
import sys
sys.path.append('./SeismicPro/')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from seismicpro.batchflow import B, V, W
from seismicpro.batchflow.models.torch import ResNet34, TorchModel
from seismicpro.src import SeismicDataset, FieldIndex, TraceIndex, seismic_plot
from seismicpro.models.metrics import PickingMetrics

Datasets

We will train a model using raw traces from one of the company's anonymized surveys:


In [ ]:
path_train = '/data/day1/data1.sgy'
markup_path='/data/day1/data1.csv'
index = FieldIndex(name='raw', path=path_train, markup_path=markup_path)

index.head()

FIRST_BREAK_TIME 	TraceNumber 	TRACE_SEQUENCE_FILE 	file_id
			raw 	raw
FieldRecord 				
16005 	74.997650 	2268 	1677 	/data/day1/data1.sgy
16005 	72.829834 	2269 	1678 	/data/day1/data1.sgy
16005 	114.031898 	2267 	1679 	/data/day1/data1.sgy
16005 	103.484001 	2270 	1680 	/data/day1/data1.sgy
16005 	136.032379 	2266 	1681 	/data/day1/data1.sgy

Load one seismogram and coresponding picking.



In [ ]:
batch = (SeismicDataset(index).next_batch(batch_size=10)

        .load(components='raw', fmt='segy')

        .load(components='markup', fmt='picks'))

In [ ]:
Draw the whole seismogram and 10 zoomed traces with labeled picking too see the pattern of picking.



In [ ]:
cv = 2000

(batch.seismic_plot('raw', index.indices[0], src_picking='markup', cmap='gray',

                     figsize=(15,5), vmax=cv, vmin=-cv, s=5, scatter_color='r')

      .seismic_plot('raw', index.indices[0], src_picking='markup', cmap='gray',

                    figsize=(15,5), vmax=cv, vmin=-cv, s=70, scatter_color='r',

                    wiggle=True, xlim=(30,40), ylim=(100,200), std=0.1))

Now we can split data to train and test parts:


In [ ]:
index.split(0.8, shuffle=42)

train_data = SeismicDataset(TraceIndex(index.train))

test_data = SeismicDataset(TraceIndex(index.test))

print('Traces in train: {}'.format(len(train_data)))

print('Traces in test: {}'.format(len(test_data)))

Model architecture

We are using simple VGG-like CNN with encoder 4 blocks and head.

Model's configuration:

    encoder block: Conv1d - downsample - Relu
    head: global pooling - dropout - fully-connected

Layes parameters:

Conv1d(body):

    filters = [8, 16, 32, 64]
    kernel_size = 3
    padding = 'same'

Dropout:

    dropout_rate = 0.1

Fully-connected:

    units = 1

Activation:

    activation = Relu

Here is a config for regression model:


In [ ]:
inputs_config = {
    'raw': {'shape': (1, W(B('raw')).shape[2])}, 
    'targets': {'shape': (1,)}
    }

config = {
    'inputs': inputs_config,
    'initial_block/inputs': 'raw',
    'body': dict(layout='cpa'*4, filters=[8, 16, 32, 64], kernel_size=3),
    'head': dict(layout='Vdf', dropout_rate=.1, units=1),
    'optimizer': dict(name='SGD', lr=0.01),
    'loss': 'l1',
    'device': 'gpu:0',
}

Training

Definition of batch size and number of training iterations:


In [ ]:
BATCH_SIZE = 64

N_ITERS = 4000

Training pipeline actions:

    Initialize model
    Load raw traces and labels.
    Normalize the traces to the zero mean and unit variance.
    Preprocess the batch of traces to make it compatible with torch models.
    Perform training step

Set the train pipeline.


In [ ]:
train_pipeline = (train_data.p
                      .init_model('dynamic', TorchModel, 'my_model', config=config)
                      .init_variable('loss', [])
                      .load(components='raw', fmt='segy')
                      .load(components='markup', fmt='picks')
                      .standardize(src='raw', dst='raw')
                      .apply_transform_all(src='raw', dst='raw', func=lambda x: np.stack(x))
                      .apply_transform_all(src='markup', dst='markup', func=lambda x: np.stack(x).astype(np.float32))
                      .train_model('my_model', B('raw'), B('markup'), fetches='loss', save_to=V('loss', mode='a'))
                      .run_later(BATCH_SIZE, n_iters=N_ITERS, drop_last=True, shuffle=True, bar=True)
                 )

It is also possible to use n_iters parameter instead of n_epochs to specify how many batches you want to feed to the model.

Run the train pipeline.



In [ ]:
train_pipeline.run(bar_desc=W(V('loss')[-1].format('Loss is: {:7.7}')))

Loss function plot:


In [ ]:
loss = train_pipeline.get_variable('loss')

plt.figure(figsize=(15,8))

plt.grid(True)

plt.xlabel("Iterations"), plt.ylabel("Loss")

plt.plot(loss)

In [ ]:
Inference

Inference pipeline is similar to the training pipeline:


In [ ]:
test_pipeline = (test_data.p
                      .import_model('my_model', train_pipeline) 
                      .init_variable('targets', [])
                      .init_variable('traces', [])
                      .init_variable('predictions', [])
                      .load(components='raw', fmt='segy')
                      .load(components='markup', fmt='picks')
                      .add_components(components='predictions')
                      .standardize(src='raw', dst='raw')
                      .apply_transform_all(src='raw', dst='raw', func=lambda x: np.stack(x))
                      .apply_transform_all(src='markup', dst='markup', func=lambda x: np.stack(x))
                      .predict_model('my_model', B('raw'), fetches='predictions', save_to=B('predictions', mode='a'))
                      .update(V('traces', 'a'), B('raw'))
                      .update(V('targets', 'a'), B('markup'))
                      .update(V('predictions', 'a'), B('predictions'))
                      .run_later(2000, n_epochs=1, drop_last=False, shuffle=False, bar=True))

In [ ]:
Run the inference pipeline on test part of train data:



In [ ]:
test_pipeline.run()

In [ ]:
Postprocessing results by concatinating all traces and pickings:



In [ ]:
preds = np.concatenate(test_pipeline.get_variable('predictions'))

targets = np.concatenate(test_pipeline.get_variable('targets'))

traces = np.squeeze(np.concatenate(test_pipeline.get_variable('traces')))

In [ ]:
print('Preds shape: {}'.format(preds.shape))
print('Targets shape: {}'.format(targets.shape))
print('Traces shape: {}'.format(traces.shape))

In [ ]:
Model evaluation

Creating an object to calculate metrics:



In [ ]:
metrics = PickingMetrics(targets, preds)

In [ ]:
Model evaluation

Creating an object to calculate metrics:



In [ ]:
metrics = PickingMetrics(targets, preds)

In [ ]:
print('MAE on test: {0:.3f}'.format(metrics.evaluate('mae')))

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Absolute error distribution')
plt.xlabel('Error, samples')
plt.ylabel('Number of traces')
_ = plt.hist(abs(targets - preds), range=(0,200), bins=100)

In [ ]:
Visual estimation



In [ ]:
subset = slice(0,400)

pts_pred = (range(len(preds[subset])), preds[subset]/2)

pts_true = (range(len(targets[subset])), targets[subset]/2)

Draw the seismogram with predictions (blue) and targets (red).


In [ ]:
cv = 1

seismic_plot(traces[subset], cmap='gray', vmax=cv, vmin=-cv, pts=pts_pred,

             s=20, scatter_color='b', figsize=(15, 5), names=['Predictions'])

seismic_plot(traces[subset], cmap='gray', vmax=cv, vmin=-cv, pts=pts_true,

             s=20, scatter_color='r', figsize=(15, 5), names=['Targets'])

Running time

System config:

    GPU: GTX GeForce 2080ti
    CPU: Intel Xeon E5-2630

Time performance:

    Model training iteration with batch size = 64: 0.05 sec.
    Inference iteration with batch size = 1000: 0.12 sec.

Criticism
Summary
Suggestions for improvement
